In [3]:
import os
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import col
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.types import StringType
from pyspark.sql.types import FloatType

In [5]:
conf = pyspark.SparkConf()

In [6]:
spark = SparkSession.builder.appName("Transformation").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/22 17:31:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/22 17:31:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
df1=spark.read.parquet("/Users/abhishek/Documents/load/Parquet/*")

In [10]:
df1.createOrReplaceTempView("Load")

# Removing bot activities - Activity 1

In [11]:
df_stage_1 = spark.sql("SELECT * FROM Load Where buyer NOT IN \
                       (SELECT buyer FROM (SELECT buyer, count(buyer) as counter from Load \
                       GROUP BY buyer, (EXTRACT(HOUR FROM block_time)), (EXTRACT(MINUTE FROM block_time))) temp\
                       WHERE counter > 1)")

In [13]:
df_stage_1.createOrReplaceTempView("Load1")

# Removing bot activities - Activity 2

In [14]:
df_stage_2 = spark.sql("SELECT * FROM Load1 Where buyer NOT IN \
                       (SELECT buyer FROM(SELECT a.buyer, a.original_amount, b.original_amount,(b.block_time - a.block_time) time_gap \
                        FROM Load1 a INNER JOIN Load1 b \
                        ON (a.buyer = b.seller AND a.block_time < b.block_time AND a.nft_token_id = b.nft_token_id )) \
                        WHERE (EXTRACT(MINUTES FROM time_gap)) < 1 AND (EXTRACT(HOURS FROM time_gap)) = 0 AND \
                       (EXTRACT(DAYS FROM time_gap)) = 0 ORDER BY time_gap ASC)")

In [14]:
df_stage_2.write.parquet("/Users/abhishek/Documents/Cleansed/Parquet")

23/03/21 04:57:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
23/03/21 04:57:29 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
23/03/21 04:57:31 WARN MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
